In [1]:
## This notebook ingest the raw "races" oracle file from the bronze volume to bronze table
### Ingest races.csv file

In [1]:
# Define parameters (can set parameters in a workflow job)
target_type   =oidlUtils.parameters.getParameter("TARGET_TYPE", "table")
target_format =oidlUtils.parameters.getParameter("TARGET_FORMAT", "delta")
bronze_folder_path    = "/Volumes/f1_bronze/bronze/f1_bronze_volume"
bronze_catalog    = "f1_bronze"
bronze_schema     = "bronze"
bronze_table_dlt = "f1_races_dlt"
bronze_table_par = "f1_races_par"


# ----------
##### Step 1 - Read the CSV file using the spark dataframe reader
#  ----------

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType


In [1]:
races_schema = StructType([
    StructField("RACEID", IntegerType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("ROUND", IntegerType(), True),
    StructField("CIRCUITREF", StringType(), True), # renamed circuit_id to match silver layer
    StructField("NAME", StringType(), True),
    StructField("DATE", StringType(), True),  # convert to DateType later
    StructField("TIME", StringType(), True),    # convert to TimeType Later
    StructField("URL", StringType(), True)
])

In [1]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv(f"{bronze_folder_path}/races.csv")

# ----------
##### Step 2 - Write the output to bronze catalog as parquet file or delta/parquet table
#  ----------

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        races_df.write.mode("overwrite").parquet(f"{bronze_folder_path}/races_par")
elif target_type == 'table':
    if target_format == 'parquet':
        races_df.write.mode("overwrite").format("parquet").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        races_df.write.mode("overwrite").format("delta").saveAsTable(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")

In [1]:
if target_type == 'file':
    if target_format == 'parquet':
        races_read_df = spark.read.parquet(f"{bronze_folder_path}/races_par")
elif target_type == 'table':
    if target_format == 'parquet':
        races_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_par}")
    elif  target_format == 'delta':
        races_read_df = spark.read.table(f"{bronze_catalog}.{bronze_schema}.{bronze_table_dlt}")